## Woermann Brock 

Serves as a normalization script before the execution of the preload.

In [11]:
import pandas as pd
import re
import datetime
import os

from datetime import datetime
from sftp_downloader import SFTPDownloader

In [12]:
folder_path = r'R:\RawData\Elite Star\\Woermann Brock\ToUpload'
done_path = r'R:\RawData\Elite Star\\Woermann Brock\\Done'

In [ ]:
def sort_files_by_date(file_path):
  files = os.listdir(file_path)
  files.sort(key=lambda x: os.path.getmtime(os.path.join(file_path, x)))
  return files

In [14]:
def extract_date_from_filename(filename):
    match = re.search(r'(\d{2})(\d{2})(\d{4})', filename)
    if match:
        date_str = match.group(1) + match.group(2) + match.group(3)
        date_obj = datetime.strptime(date_str, '%d%m%Y')
        return date_obj
    else:
        return datetime.min # If the date is not found, set it to a default value

file_list = os.listdir(folder_path)
file_list.sort(key=lambda x: extract_date_from_filename(x), reverse=True)

sorted_files = [{'File Name': filename, 'Date': extract_date_from_filename(filename)} for filename in file_list]

df = pd.DataFrame(sorted_files)
print(df)

                 File Name                 Date                   Latest
0  ESTexport-14052024 .txt  2024-05-14 00:00:00  ESTexport-02052024 .txt
1  ESTexport-13052024 .txt  2024-05-13 00:00:00  ESTexport-02052024 .txt
2  ESTexport-12052024 .txt  2024-05-12 00:00:00  ESTexport-02052024 .txt
3  ESTexport-11052024 .txt  2024-05-11 00:00:00  ESTexport-02052024 .txt
4  ESTexport-10052024 .txt  2024-05-10 00:00:00  ESTexport-02052024 .txt
5  ESTexport-09052024 .txt  2024-05-09 00:00:00  ESTexport-02052024 .txt
6  ESTexport-08052024 .txt  2024-05-08 00:00:00  ESTexport-02052024 .txt
7  ESTexport-07052024 .txt  2024-05-07 00:00:00  ESTexport-02052024 .txt
8                  Back up  0001-01-01 00:00:00  ESTexport-02052024 .txt


In [ ]:
def normalize_filename(filename):
    base, ext = os.path.splitext(filename)
    normalized_filename = f"{base.strip()} {ext.strip()}"
    return normalized_filename

file_data = []
for filename in os.listdir(folder_path):
    if os.path.isfile(os.path.join(folder_path, filename)):
        normalized_filename = normalize_filename(filename)
        if normalized_filename != filename:  
            file_data.append([filename, normalized_filename])
            destination_path = os.path.join(folder_path, normalized_filename)
            if os.path.exists(destination_path):
                os.remove(destination_path)  # Remove the existing file
            os.rename(os.path.join(folder_path, filename), destination_path)

df = pd.DataFrame(file_data, columns=['Initial Filename', 'Final Filename'])
print(df)


Empty DataFrame
Columns: [Initial Filename, Final Filename]
Index: []
